In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, "/home/amir/work/mps/src/")

using ExactDiagonalizationTools
using MatrixProductStateTools
using TensorNetAlgs
using Plots

In [ ]:
lx = 31
delta = 2. #ratio of Jz to Jp
r = 0.6
maxdim = 40
dt = 0.1

samples=10
nsteps = 40
nsamples = 10

In [ ]:
mpo = xxzlong_mpo(ComplexF64, lx, 2, delta, r)
sz = Matrix(sz_half)

ts = 0:Int(nsteps*dt)

alldata = []
for sample in 1:nsamples
    conf = rand(0:1, lx)
    for k in [0,1]
        zdata = Vector{Float64}[]
        conf[div(lx, 2)+1] = k
        mps = MatrixProductState{ComplexF64}(lx, 2, conf, maxdim, 1.0e-6)
        #normalize!(mps)
        push!(zdata, (-1)^k .* real.(measure_1point(mps, sz)))
        println("Sample $sample, spin $k, time : 0.0")

        mpoc = convert(MatrixProductOperator{ComplexF64}, mpo)
        env = initialenv(mps, mpoc)

        for t in dt:dt:nsteps*dt
            tdvp1sitesweep!(dt, mps, mpoc, env, verbose=false)
            if abs(round(t)-t) ≈ 0.
                push!(zdata, real.(measure_1point(mps, sz)))
                println("Sample $sample, spin $k, time : $t")
            end
        end
        push!(alldata, zdata)
    end
end